In [135]:
%cd code

[Errno 2] No such file or directory: 'code'
/opt/ml/project/team/code


In [234]:
import json
from collections import defaultdict
from data_utils import convert_state_dict
import glob

In [235]:
ROOT = '/opt/ml/project/team/code'
PRED_ROOT = f'{ROOT}/ensemble/pred_data/SUMBT'
ENSEMBLE_ROOT = f'{ROOT}/ensemble'


files = glob.glob(f'{PRED_ROOT}/*.csv')
# files = ['/opt/ml/project/team/code/predictions/66_dec_long.csv']
# ensemble 결과 
output_name = 'coco_plus_sumbt.csv'

In [236]:
slot_meta = json.load(open(f"/opt/ml/input/data/train_dataset/slot_meta.json"))

In [237]:
!ls /opt/ml/input/data

coco_comb  eval_dataset  tmp  train_dataset


In [238]:
pred_jsons = [json.load(open(file, 'r')) for file in files]

In [239]:
def add_none(state_dict):
    for k in slot_meta:
        if k not in state_dict:
            state_dict[k] = 'none'
    return state_dict

def remove_none(state_dict):
    res = {}
    for k, v in state_dict.items():
        if v == 'none':
            continue
        res[k] = v
    return res

In [240]:
def postprocess_state(state):
    for i, s in state.items():
        s = s.replace(" : ", ":")
        s = s.replace(" & ", "&")
        s = s.replace(" = ", "=")
        s = s.replace("( ", "(")
        s = s.replace(" )", ")")
        state[i] = s.replace(" , ", ", ")
    return state

In [241]:
state_dict_jsons = []
for pred_json in pred_jsons:
    cur_pred = {}
    for guid, v in pred_json.items():
        tmp = convert_state_dict(v)
        tmp = postprocess_state(tmp)
        tmp = add_none(tmp)
        cur_pred[guid] = tmp
    state_dict_jsons.append(cur_pred)

In [242]:
guid_preds = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
for cur_json in state_dict_jsons:
    for guid, slot_pred in cur_json.items():
        for slot, slot_value in slot_pred.items():
            guid_preds[guid][slot][slot_value] += 1

In [243]:
ensemble_preds = defaultdict(dict)
for guid, guid_pred in guid_preds.items():
    for slot, slot_values in guid_pred.items():
        ans = max(slot_values, key=lambda x: slot_values[x])
        ensemble_preds[guid][slot] = ans

In [244]:
final_dict_preds  = {}
for guid, guid_pred in ensemble_preds.items():
    guid_pred = remove_none(guid_pred)
    final_dict_preds[guid] = guid_pred

In [245]:
final_preds  = {}
for guid, guid_pred in final_dict_preds.items():
    tmp = []
    for k, v in guid_pred.items():
        tmp.append(f'{k}-{v}')
    final_preds[guid] = tmp

In [246]:
json.dump(
    final_preds,
    open(f"{ENSEMBLE_ROOT}/{output_name}", "w"),
    indent=2,
    ensure_ascii=False,
)

## diff by turn slot

In [154]:
diff = 0
total = 0
for guid, guid_pred in guid_preds.items():
    for slot, slot_values in guid_pred.items():
        if len(slot_values) != 1:
            diff += 1
            break
        total += 1
    if diff > 1:
        break

In [155]:
slot_values

defaultdict(int, {'서울 남쪽': 1, '서울 중앙': 2})

In [156]:
max(slot_values, key=lambda x: slot_values[x])

'서울 중앙'

In [187]:
guid = 'square-silence-9151:숙소_관광_11-0'

In [191]:
pj

['관광-이름-국립현대미술관 서울관',
 '관광-종류-박물관',
 '관광-지역-서울 북쪽',
 '숙소-가격대-dontcare',
 '숙소-스파 유무-yes',
 '숙소-예약 기간-1',
 '숙소-예약 명수-2',
 '숙소-예약 요일-화요일',
 '숙소-이름-더 그린',
 '숙소-조식 가능-yes',
 '숙소-종류-에어비엔비',
 '숙소-지역-dontcare',
 '식당-가격대-적당',
 '식당-야외석 유무-yes',
 '식당-예약 명수-2',
 '식당-예약 시간-20:40',
 '식당-예약 요일-화요일',
 '식당-이름-어차피자',
 '식당-종류-양식당',
 '식당-지역-서울 북쪽']

In [188]:
final_preds[guid]

['숙소-가격대-적당', '숙소-스파 유무-yes', '숙소-종류-호텔']

In [190]:
fp

['관광-이름-국립현대미술관 서울관',
 '관광-종류-박물관',
 '관광-지역-서울 북쪽',
 '숙소-가격대-dontcare',
 '숙소-스파 유무-yes',
 '숙소-예약 기간-1',
 '숙소-예약 명수-2',
 '숙소-예약 요일-화요일',
 '숙소-이름-더 그린',
 '숙소-조식 가능-yes',
 '숙소-종류-에어비엔비',
 '숙소-지역-dontcare',
 '식당-가격대-적당',
 '식당-야외석 유무-yes',
 '식당-예약 명수-2',
 '식당-예약 시간-20:40',
 '식당-예약 요일-화요일',
 '식당-이름-어차피자',
 '식당-종류-양식당',
 '식당-지역-서울 북쪽']

In [189]:
for guid in final_preds:
    pj = sorted(pred_jsons[0][guid])
    fp = final_preds[guid]
    if pj != fp:
        print(pj)
        print(fp)

['관광-역사적-yes', '관광-이름-서울중앙성원', '관광-종류-랜드마크', '관광-지역-dontcare', '숙소-가격대-dontcare', '숙소-예약 기간-5', '숙소-예약 명수-2', '숙소-예약 요일-목요일', '숙소-이름-서촌 한옥 게스트하우스', '숙소-종류-게스트 하우스', '숙소-지역-서울 중앙', '식당-예약 시간-16:45', '식당-예약 요일-목요일']
['관광-역사적-yes', '관광-이름-서울중앙성원', '관광-종류-랜드마크', '관광-지역-dontcare', '숙소-가격대-dontcare', '숙소-예약 기간-5', '숙소-예약 명수-2', '숙소-예약 요일-목요일', '숙소-이름-서촌 한옥 게스트하우스', '숙소-종류-게스트 하우스', '숙소-지역-서울 중앙', '식당-예약 시간-16:45']
['관광-역사적-yes', '관광-이름-서울중앙성원', '관광-종류-랜드마크', '관광-지역-dontcare', '숙소-가격대-dontcare', '숙소-예약 기간-5', '숙소-예약 명수-2', '숙소-예약 요일-목요일', '숙소-이름-서촌 한옥 게스트하우스', '숙소-종류-게스트 하우스', '숙소-지역-서울 중앙', '식당-종류-한식당', '식당-지역-서울 남쪽']
['관광-역사적-yes', '관광-이름-서울중앙성원', '관광-종류-랜드마크', '관광-지역-dontcare', '숙소-가격대-dontcare', '숙소-예약 기간-5', '숙소-예약 명수-2', '숙소-예약 요일-목요일', '숙소-이름-서촌 한옥 게스트하우스', '숙소-종류-게스트 하우스', '숙소-지역-서울 중앙', '식당-종류-한식당', '식당-지역-서울 중앙']
['관광-역사적-yes', '관광-이름-서울중앙성원', '관광-종류-랜드마크', '관광-지역-dontcare', '숙소-가격대-dontcare', '숙소-예약 기간-5', '숙소-예약 명수-2', '숙소-예약 요일-목요일', '숙소-이름-서촌 한옥 게스트하우스', '숙소-종류-게스트 하우스', '숙소-지역-

In [171]:
guid_preds['square-silence-9151:숙소_관광_11-0']

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {'숙소-가격대': defaultdict(int, {'적당': 3}),
             '숙소-스파 유무': defaultdict(int, {'yes': 3}),
             '숙소-종류': defaultdict(int, {'호텔': 3}),
             '관광-경치 좋은': defaultdict(int, {'none': 3}),
             '관광-교육적': defaultdict(int, {'none': 3}),
             '관광-도보 가능': defaultdict(int, {'none': 3}),
             '관광-문화 예술': defaultdict(int, {'none': 3}),
             '관광-역사적': defaultdict(int, {'none': 3}),
             '관광-이름': defaultdict(int, {'none': 3}),
             '관광-종류': defaultdict(int, {'none': 3}),
             '관광-주차 가능': defaultdict(int, {'none': 3}),
             '관광-지역': defaultdict(int, {'none': 3}),
             '숙소-도보 가능': defaultdict(int, {'none': 3}),
             '숙소-수영장 유무': defaultdict(int, {'none': 3}),
             '숙소-예약 기간': defaultdict(int, {'none': 3}),
             '숙소-예약 명수': defaultdict(int, {'none': 3}),
             '숙소-예약 요일': defaultdict(int, {'none': 3}),
        